In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
from geopandas import GeoSeries
from shapely.geometry import Point, LineString

1. Import the data located at this link. It has information on people infected with dengue at the district level for 2015 to 2021.

In [2]:
base_dengue = pd.read_csv(r'../../_data/data_dengue_peru.csv',thousands=',', na_values=['NaN', 'nan'], dtype={'Casos': 'float'})

2. Generate ubigeo for Departments and Provinces taking the first two and four numbers. Hint: Use this code.

In [3]:
# Create 'Ubigeo_Departamento', 'Ubigeo_Provincia' and rename 'Ubigeo' to 'Ubigeo_Distrito'
base_dengue['Ubigeo_Departamento'] = base_dengue['Ubigeo'].astype(str).str[:2]
base_dengue['Ubigeo_Provincia'] = base_dengue['Ubigeo'].astype(str).str[:4]
base_dengue = base_dengue.rename(columns={'Ubigeo':'Ubigeo_Distrito'})

In [4]:
# Create data frames 'base_Distrito', 'base_Provincia', and 'base_Departamento' to facilitate the creation of the next plots

# Define the levels we are going to group the columns 'Casos'
nivel = ['Distrito', 'Provincia', 'Departamento']
# Dictionary to store the results
datas = {}
# Loop to create the data frames
for nivel in nivel:
    columna_ubigeo = f'Ubigeo_{nivel}'
    datas[nivel] = base_dengue.groupby([columna_ubigeo, 'Año', nivel ])['Casos'].sum(min_count=0).reset_index()
    
    exec(f"base_{nivel} = datas['{nivel}']")

In [6]:
# Get geometry information
maps = gpd.read_file(r'../../_data/shape_file/DISTRITOS.shp')

In [11]:
# Keeps geometry information for 'Distrito'
maps_DIST = maps[['IDDIST', 'geometry']]
maps_DIST = maps_DIST.rename({'IDDIST':'Ubigeo_Distrito'}, axis =1 )
maps_DIST['Ubigeo_Distrito'] = maps_DIST['Ubigeo_Distrito'].astype(str)

# Keeps geometry information for 'Provincia'
maps_PROV = maps[['IDPROV', 'geometry']]
maps_PROV = maps_PROV.rename({'IDPROV':'Ubigeo_Provincia'}, axis =1 )
maps_PROV['Ubigeo_Provincia'] = maps_PROV['Ubigeo_Provincia'].astype(str)

# Keeps geometry information for 'Departamento'
maps_DEP = maps[['IDDPTO', 'geometry']]
maps_DEP = maps_DEP.rename({'IDDPTO':'Ubigeo_Departamento'}, axis =1 )
maps_DEP['Ubigeo_Departamento'] = maps_DEP['Ubigeo_Departamento'].astype(str)

3. Use geopandas to plot the number of cases in 2021 by the district using a continuous legend. Do not forget to indicate the color of NA values. Use this shapefile.